In [127]:
from autodp.mechanism_zoo import ExactGaussianMechanism, PureDP_Mechanism
from autodp.transformer_zoo import Composition
import matplotlib.pyplot as plt
from sklearn.datasets import load_svmlight_file

import time
import random
import numpy as np
from pathlib import Path
from phe import paillier

In [107]:
DATA_PATH = Path('D://dataset')

def read_libsvm(dname):
    if (DATA_PATH / f'{dname}.txt').exists():
        xs, ys = load_svmlight_file(str((DATA_PATH / f'{dname}.txt')))
        pt = 0.2 if dname != 'iris' else 0.1
        xs, xs_t, ys, ys_t = train_test_split(xs.toarray(), ys, train_size=0.2)
        return xs, ys, xs_t, ys_t
    else:
        dir = DATA_PATH / dname
        xs, ys = load_svmlight_file(str(dir / f'{dname}.txt'))
        xs_t, ys_t = load_svmlight_file(str(dir / f'{dname}.t'))
        return xs.toarray(), ys, xs_t.toarray(), ys_t

class OPE:
    def __init__(self):
        self.a = random.randint(1, 1<<16)
        self.b = random.randint(1, 1<<16)
        self.c = random.randint(1, 1<<16)

    def __call__(self, x):
        res = self.a * (x - self.c) ** 3 + self.b
        if x >= self.c:
            return res
        return 2 * self.b - res
        return 1.5 * x + 2

class Phaser:
    def __init__(self, arr, p):
        m = p * 0.5
        # m = 0.1
        self.mi = min(arr) - m
        self.ma = max(arr) + m
        self.p = p

    def __call__(self, x):
        x = x + self.p
        if x > self.ma:
            return x + self.mi - self.ma
        elif x < self.mi:
            return x + self.ma - self.mi
        else:
            return x

def trans_phase(ds, i):
    A = [x[i] for x in ds[0]]
    r = random.random() * 0.4 + 0.4
    # r = 0.8
    pa = Phaser(A, (max(A)-min(A)) * r)
    for x in ds[0]:
        x[i] = pa(x[i])
    for x in ds[2]:
        x[i] = pa(x[i])

def trans_ope(ds):
    nf = len(ds[0][0])
    ope = OPE()
    for x in ds[0]:
        for i in range(nf):
            x[i] = ope(x[i])
    for x in ds[2]:
        for i in range(nf):
            x[i] = ope(x[i])

In [113]:
sigma1 = 5.0
sigma2 = 8.0

gm1 = ExactGaussianMechanism(sigma1,name='GM1')

compose = Composition()
composed_mech = compose([gm1], [3])

delta1 = 1e-4

In [126]:
# ds = read_libsvm('dna')
# xs = [i[0] for i in ds[0]]
xs = np.random.uniform(-10000, 10000, 1000)

In [124]:
# DP

st = time.time()

for x in xs:
    eps1 = composed_mech.get_approxDP(delta1)
    x += eps1

ed = time.time()

print(f'{(ed-st)*1000:.3f}')

1609.018


In [125]:
# Phaser

st = time.time()

pa = Phaser(xs, (max(xs)-min(xs)) * 0.5)
ope = OPE()
sorted(xs)
for x in xs:
    x = ope(pa(x))

ed = time.time()

print(f'{(ed-st)*1000:.3f}')

12.006


In [128]:
# HE

st = time.time()

pk, sk = paillier.generate_paillier_keypair()
for x in xs:
    _x = pk.encrypt(x)

ed = time.time()

print(f'{(ed-st)*1000:.3f}')

185922.755
